# MNIST DATA SET

- 주로 CNN에서 좋은 성능을 보임
- 그 이유는 이미지 데이터의 공간적 구조를 효과적으로 학습할 수 있기 때문
- MLP: Fully Connected Layer를 사용해 입력 이미지를 1D 벡터로 변환해 학습함
- 하지만 CNN은 합성곱 연산(Convolution)을 사용하여 지역적 특징(Local Patterns)을 학습하고, 풀링(Pooling) 연산을 통해 불필요한 정보를 제거하면서도 중요한 특징을 유지할 수 있음
- Ex. MNIST 숫자 '3'의 픽셀 위치가 조금 이동했다면
  - MLP: 모든 픽셀을 일렬로 펼쳐 학습하므로, '3'의 위치가 조금만 달라져도 같은 숫자로 인식하지 못할 수 있음
  - CNN: 합성곱 필터가 특정 패턴을 학습하므로, '3'의 위치가 달라져도 같은 숫자로 인식 가능

## CNN의 핵심 메커니즘

1) 합성곱(Convolution) 연산
- CNN의 핵심은 작은 필터(커널)을 사용하여 지역적인 특징을 학습하는 것
- 엣지(Edge), 코너(Corner), 선(Line) 등을 감지할 수 있음
- CNN은 계층을 쌓으면서 더 복잡한 특징을 학습함
  - 초기 계층 -> 단순한 패턴(엣지, 코너)
  - 중간 계층 -> 숫자의 일부 패턴 학습
  - 최종 계층 -> 숫자 전체 형태 학습

2) 풀링(Pooling) 연산
- 최대 풀링(Max Pooling) 또는 평균 풀링(Average Pooling)을 통해 불필요한 정보 제거 & 중요한 특징 유지
- 공간 정보를 유지하면서도 모델이 더 일반화될 수 있도록 돕는다
- 위치 변화에 대한 불변성(Translation Invariance)을 증가시킨다

3) 가중치 공유(Weight Sharing)
- 일반적인 MLP에는 모든 뉴런이 고유한 가중치(Weight)를 학습하지만, CNN에서는 필터가 모든 영역에서 공유된다.
- 이는 파라미터 수를 대폭 감소시키고 연산 효율을 높이며 과적합(Overfitting)도 방지할 수 있다.

**즉, CNN은 이미지의 공간 구조를 그대로 학습할 수 있어 성능이 우수함**

## 주요 연구 내용

(1) LeCun et al. (1998) - “Gradient-Based Learning Applied to Document Recognition”
- Yann LeCun이 제안한 LeNet-5 모델 소개하는 논문
- CNN이 MNIST에서 뛰어난 성능 보임
- 합성곱 계층(Convolutional Layer)과 풀링 계층(Pooling Layer)을 조합하여 특징 효과적으로 추출
- 기존 MLP와 비교해 더 높은 정확도와 일반화 성능 달성

(2) Krizhevsky et al. (2012) - “ImageNet Classification with Deep Convolutional Neural Networks” (AlexNet)
- AlexNet을 제안하며, CNN이 대규모 이미지 분류에서 뛰어난 성능을 보임
- 더 복잡한 이미지에서도 CNN이 효과적
- ReLU 활성화 함수 적용해 비선형 표현력 강화
- 드롭아웃(Dropout) 기법 사용해 과적합 방지
- 합성곱 연산 깊이 쌓아 학습 성능 극대화

(3) He et al. (2016) - “Deep Residual Learning for Image Recognition” (ResNet)
- Residual Learning(잔차 학습) 기법을 도입하여 깊은 신경망의 학습 문제를 해결
- MNIST 같은 간단한 데이터뿐만 아니라, 복잡한 이미지 데이터에서도 CNN이 강력함을 입증
- ResNet 블록을 사용하면 매우 깊은 CNN 네트워크도 효과적으로 학습 가능
- 일반적인 CNN 모델(LeNet-5)보다 더 높은 정확도를 달성 가능
- MNIST에서도 ResNet 적용하면 정확도 99.5% 이상으로 향상됨

따라서 본 코드에서는
- Residual Block을 사용
- Batch Normalization & Dropout 추가
- ReLU 활성화 함수 적용
- Adam 옵티마이저 사용
- Data Augmentation을 활용하고자 함

# Library Load

In [22]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.optim as optim

# Data Preprocessing & Data Augmentation

- 이미지 회전 -> 모델이 다양한 방향의 숫자를 인식할 수 있도록
- 랜덤한 기하학적 변환 -> 모델이 다양한 필기 스타일을 인식할 수 있도록
- ToTensor -> 흑백 이미지(1채널), 각 픽셀 값[0, 255] 범위 갖는데, 이를 [0, 1]로 변환
- Normalize -> 신경망 학습이 더 빠르고 안정적으로 이뤄지기 위함과 Gradient Vanishing 문제 방지(값이 너무 크거나 작으면 학습 어려움)

In [13]:
transform_train = transforms.Compose([
    transforms.RandomRotation(10),  # 데이터 증강: 회전
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # 데이터 증강: 기하학적 변환
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Data Load

- 훈련 데이터: 데이터 증강 적용
- 테스트 데이터: 증강 없이 원본 이미지 정규화만 수행
- DataLoader 사용해 데이터 배치 처리 및 샘플링 수행
  - 훈련과 테스트 데이터를 미니배치 단위로 나눠 처리
  - batch_size= 한번에 64개 이미지 가져와서 학습
  - shuffle=true: 데이터 순서 랜덤하게 섞어 데이터 순서에 의존하지 않도록 과적합 방지
  - 매 Epoch마다 랜덤하게 섞어 일반화 성능 향상
- 테스트셋
  - 모델 평가 시 연산 속도 최적화하기 위해 큰 배치 크기 사용
  - 모델 평가에서는 가중치 업데이트가 없어 큰 배치 사용 가능
  - 테스트 데이터는 항상 같은 순서로 평가해야 평가 결과 일관됨

In [14]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Residual Block

- in_channels: 입력 채널 개수
- out_channels: 출력 채널 개수
- stride: 합성곱 연산에서 사용하는 보폭(기본값 = 1)
- downsample: 입력과 출력의 크기가 다를 경우 1x1 컨볼루션 적용해 차원을 맞춤

## Residual Block 동작방식

1. 입력(x)을 그대로 유지하는 residual 생성
2. 두개의 3x3 컨볼루션 통과하며 특징 학습
3. 입력 그대로 출력과 더해 잔차 연결 적용
4. ReLU 활성화 후 최종 출력

- self.conv1, bn1: 첫번째 합성곱 연산은 3x3 커널을 사용하여 특징 추출, padding=1을 설정해 입력 크기 유지, BatchNorm이 편향 조정해 bias 불필요
- self.conv2, bn2: 두번째 합성곱 연산도 커널 3x3 사용하고 stride =1로 크기 유지하면서 특징 추출, Bachnorm 적용
- 입력 출력의 크기가 다를 경우 다운샘플링 사용해 크기 맞춤
- 순전파는 입력 데이터 residual에 저장해 out과 나중에 더함
- 첫번째 합성곱 거쳐 첫번째 특징 맵 생성 후 BatchNorm 적용 > ReLU로 활성화해 비선형성 추가
- 두번재는 ReLU 적용 안하고 다음 잔차 연결에서 적용
- ReLU 마지막으로 한번 더 적용해 비선형성 추가

In [17]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        if self.downsample:
            residual = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += residual  # 잔차 연결 (Residual Connection)
        out = self.relu(out)
        return out

# ResNet 모델 구현

- conv1: 입력 채널은 1, 출력 채널은 64(흑백이기 때문)
- 첫번째 합성곱층에서는 16~64개 정도 필터 사용
- conv2: stride=2를 적용해 공간적 차원 축소(높이, 너비) -> 추상적 특징 학습
- avg_pool: Adaptive Average Pooling 사용해 입력 크기에 상관없이 마지막 특징 맵 1x1 크기로 축소 -> 클래스 분류 문제에 맞춰 최종 출력 생성
- fc: Flatten 된 특징을 받아 최종적으로 클래스 개수로 매핑하는 완전 연결 레이어
- make_layer로 Residual Block들을 하나의 계층으로 묶어줌


## 순전파와 역전파 개념

	-	순전파(Forward Propagation): 입력 데이터를 받아 신경망을 거쳐 최종 출력을 계산하는 과정
	-	역전파(Backward Propagation): 모델이 예측한 값과 실제 값의 차이(손실, Loss)를 이용하여 가중치를 업데이트하는 과정

  순전파 과정
  1.	입력 데이터(x)가 신경망을 통과하며 변환됨
  2.	합성곱 레이어(Conv2D), 활성화 함수(ReLU), BatchNorm, 풀링(Pooling) 등의 연산을 거침
  3.	완전 연결 레이어(FC)를 거쳐 최종 예측 값(output)을 생성
  4.	출력 값(output)이 손실 함수(Loss Function)로 전달됨

  역전파 과정
  1.	손실 함수(Loss Function): 예측 값과 실제 값의 차이를 계산
	2.	기울기(Gradient) 계산: 각 가중치(Weight)에 대한 손실의 기울기를 자동으로 계산 (loss.backward())
	3.	가중치 업데이트: 옵티마이저(Optimizer)를 사용하여 가중치를 업데이트 (optimizer.step())

In [19]:
class ResNetMNIST(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNetMNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)  # MNIST는 흑백(1채널)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(64, 64, 2)
        self.layer2 = self.make_layer(64, 128, 2, stride=2)
        self.layer3 = self.make_layer(128, 256, 2, stride=2)
        self.layer4 = self.make_layer(256, 512, 2, stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, in_channels, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride, downsample))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# 학습 및 평가 함수

In [20]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Epoch {epoch} [{batch_idx}/{len(train_loader)}] Loss: {loss.item():.4f}")

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Test Accuracy: {accuracy:.2f}%")

# 최적 모델 선택 및 학습

- CrossEntropyLoss는 다중 클래스 분류 문제에서 사용하는 손실 함수
- 소프트맥스 확률값과 실제 정답을 비교하여 손실을 계산함
예측값과 실제값을 비교해 손실을 줄이는 방향으로 가중치 업데이트

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetMNIST().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 학습 진행

In [24]:
for epoch in range(1, 10):  # 10 Epoch 학습
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader)

Epoch 1 [0/938] Loss: 2.5245
Epoch 1 [100/938] Loss: 0.1690
Epoch 1 [200/938] Loss: 0.1130
Epoch 1 [300/938] Loss: 0.1122
Epoch 1 [400/938] Loss: 0.1400
Epoch 1 [500/938] Loss: 0.0725
Epoch 1 [600/938] Loss: 0.0727
Epoch 1 [700/938] Loss: 0.0919
Epoch 1 [800/938] Loss: 0.1318
Epoch 1 [900/938] Loss: 0.1047
Test Accuracy: 98.58%
Epoch 2 [0/938] Loss: 0.0277
Epoch 2 [100/938] Loss: 0.0463
Epoch 2 [200/938] Loss: 0.0566
Epoch 2 [300/938] Loss: 0.0477
Epoch 2 [400/938] Loss: 0.0456
Epoch 2 [500/938] Loss: 0.2124
Epoch 2 [600/938] Loss: 0.0233
Epoch 2 [700/938] Loss: 0.0183
Epoch 2 [800/938] Loss: 0.1493
Epoch 2 [900/938] Loss: 0.1193
Test Accuracy: 99.06%
Epoch 3 [0/938] Loss: 0.1069
Epoch 3 [100/938] Loss: 0.0052
Epoch 3 [200/938] Loss: 0.1309
Epoch 3 [300/938] Loss: 0.0805
Epoch 3 [400/938] Loss: 0.3768
Epoch 3 [500/938] Loss: 0.1098
Epoch 3 [600/938] Loss: 0.0038
Epoch 3 [700/938] Loss: 0.0233
Epoch 3 [800/938] Loss: 0.0493
Epoch 3 [900/938] Loss: 0.0472
Test Accuracy: 98.91%
Epoch 4 [0

In [25]:
# 학습이 완료된 모델 저장
model_save_path = "./mnist_resnet.pth"
torch.save(model.state_dict(), model_save_path)

- 손실 2.52 -> 0.006까지 감소
- Test Accuracy 99.49%